## Mandar a Predecir Datos

Para no saturar la consulta se manda a predecir desde la hoja "predecir", esta funcion recorre resgistros por fila y consulta mediante loop

* Se debe correr sas 0 generacion datos ODD_V2.
* se deben tener los paquetes os, pandas, xlrd y request para la lectura de excel

In [1]:
#Carga de paquetes
import os
import pandas as pd
import requests

## carga de datos

In [2]:
mainpath = "/Users/salibaud/Desktop/python/kaufmann/ESTIMADOR DE KML AUTOMOVILES"
filename = "Mnadar a predecir.xlsx"
fullpath = os.path.join(mainpath, filename)
xl = pd.ExcelFile(fullpath)
xl
data = xl.parse('predecir')
data.head()

,RUT,NRO_CHASSIS,PROVINCIA_CLIENTE,FORMA_DE_PAGO,FAMILIA,MODELO,SEXO_CLIENTE,MONTO_VENTA,MONEDA,TIPO_CAMBIO,MODELO_DESCRIPCION
0,10759877-4,WDC1569051J057059,SANTIAGO,NORMAL,NGCC,CLASE GLA,Femenino,2.564100e+07,PESO,616.42,GLA 220 CDI 4MATIC
1,12486903-K,WDD1760431J510690,SANTIAGO,NORMAL,NGCC,CLASE A,Desconocido,1.469748e+07,PESO,670.16,A 200
2,15592049-1,WDC1569051J335366,SANTIAGO,NORMAL,NGCC,CLASE GLA,Desconocido,2.440534e+07,PESO,666.95,GLA 220 D
3,16313496-9,WDC1569431J143184,CHOAPA,FINANC. EXTERNO,NGCC,CLASE GLA,Femenino,3.240082e+04,DOLAR KAUFMANN,635.61,GLA 200
4,3324773-7,WDD2120481A704708,CONCEPCION,NORMAL,MID SIZE,CLASE E,Desconocido,2.176471e+07,PESO,NaN,E 200 CGI BLUE EFFICIENCY


## Llamada a la api

In [3]:

login = {
    'username': 'kaufmann_api',
    'password': 'BpZgb74qtk',
}


resultados_modelo=[]

i=0

for i in range(len(data)):
    params=data.iloc[i].to_dict()
    results = requests.post('https://api.odd.co/clairvoyant/jwt-auth/', json=login)
    token = results.json()['token']
    headers = {'Authorization': 'JWT ' + token}
    results = requests.post('https://api.odd.co/clairvoyant/kaufmann/kml/',headers=headers,json=params)
    print("predicción numero ",i)
    x=results.json()
    resultados_modelo.append(x)




predicción numero  0
predicción numero  1
predicción numero  2
predicción numero  3
predicción numero  4
predicción numero  5
predicción numero  6
predicción numero  7
predicción numero  8
predicción numero  9
predicción numero  10
predicción numero  11
predicción numero  12
predicción numero  13
predicción numero  14
predicción numero  15
predicción numero  16
predicción numero  17
predicción numero  18
predicción numero  19
predicción numero  20
predicción numero  21
predicción numero  22
predicción numero  23


## Union de resultados con datos originales

In [4]:
df = pd.DataFrame(resultados_modelo)
a=data.join(df)
a.head()

,RUT,NRO_CHASSIS,PROVINCIA_CLIENTE,FORMA_DE_PAGO,FAMILIA,MODELO,SEXO_CLIENTE,MONTO_VENTA,MONEDA,TIPO_CAMBIO,MODELO_DESCRIPCION,predictions
0,10759877-4,WDC1569051J057059,SANTIAGO,NORMAL,NGCC,CLASE GLA,Femenino,2.564100e+07,PESO,616.42,GLA 220 CDI 4MATIC,[748.5472863538897]
1,12486903-K,WDD1760431J510690,SANTIAGO,NORMAL,NGCC,CLASE A,Desconocido,1.469748e+07,PESO,670.16,A 200,[1685.2684298059164]
2,15592049-1,WDC1569051J335366,SANTIAGO,NORMAL,NGCC,CLASE GLA,Desconocido,2.440534e+07,PESO,666.95,GLA 220 D,[2966.0791811289973]
3,16313496-9,WDC1569431J143184,CHOAPA,FINANC. EXTERNO,NGCC,CLASE GLA,Femenino,3.240082e+04,DOLAR KAUFMANN,635.61,GLA 200,[1086.0797733633553]
4,3324773-7,WDD2120481A704708,CONCEPCION,NORMAL,MID SIZE,CLASE E,Desconocido,2.176471e+07,PESO,NaN,E 200 CGI BLUE EFFICIENCY,[1232.6323550841414]


## exportacion de archivo

Una vez generada la consulta, se manda a exportar a excel el listado

In [5]:
a.to_csv('resultados_prediccion.csv')